  This chapter will cover the following tasks:
  * How Python evaluates decorator syntax
  * How Python decides whether a variable is local 
  * Why closures exist and how they work
  * What problem is solved by nonlocal 
  
  With this grounding, we can tackle further decorator topics:
  * Implementing a well-behaved decorator 
  * Interesting decorators in the standard library 
  * Implementing a parameterized decorator 

### Decorator 101
The following codes has the same effect:

In [ ]:
def target():
    print('running target()')

target = decorator(target)

In [ ]:
@decorator
def target():
    print('running target()')

A decorator usually replaces a function with a different one

In [1]:
def deco(func):
    def inner():
        print('running inner()')
    return inner 

@deco 
def target():
    print('running target()')

In [2]:
target() # Invoking the decorated target actually runs inner

running inner()


In [3]:
target

<function __main__.deco.<locals>.inner()>

In [11]:
registry = []

def register(func):
    print('running register(%s)' % func) 
    registry.append(func)
    return func

@register
def f1():
    print('running f1()')

def f3():
    print('running f3()')

running register(<function f1 at 0x000002733FAAA3A0>)


The above example emphasize that function decorator executed as soon as the module is imported

In [12]:
f1() # if decorator doesn't define an inner function, run f1()

running f1()


In [13]:
f1

<function __main__.f1()>

Back to the example in Chapter 6. The repetition is problematic because someone may add a new promotional strategy function and forget to manually add it to the promos list in which case, best_promo will silently ignore the new strategy. 

In [14]:
promos = []

def promotion(pro_func):
    promos.append(pro_func)
    return pro_func 

@promotion
def fidelity(order):
    """5% discount for customers with 1000 or more fidelity points"""
    return order.total() *.5 if order.customer.fidelity >= 1000 else 0

def best_promo(order):
    """Select best discount available"""
    return max(promo(order) for promo in promos)

### Closures 
A closure is a function with an extended scope that encompasses nonglobal variables referenced in teh body of the function but not defined there.

In [16]:
class Average():
    def __init__(self):
        self.series = []
    def __call__(self,new_value): #The Average class creates instances that are callable
        self.series.append(new_value)
        total = sum(self.series)
        return total/len(self.series)


In [17]:
avg = Average()
avg(10)

10.0

In [18]:
avg(11) #agv keep the history of previous values

10.5

higher-order function

In [19]:
def make_averager():
    series = []
    def averager(new_value):
        series.append(new_value)
        total = sum(series)
        return total/len(series)
    return averager 

In [20]:
avg = make_averager()
avg(10)

10.0

In [21]:
avg(11)

10.5

In [22]:
series

NameError: name 'series' is not defined

Note that series is a local variable of make_averager. Within averager, series is a free variable. This is a technical term meaning a variable that is not bounding in the local scope

Inspecting the returned averager object shows how Python keeps the names of local and free variables in the /_/_code/_/_ attribute that represents the compiled body of the function.

In [23]:
avg.__code__.co_varnames

('new_value', 'total')

In [24]:
avg.__code__.co_freevars

('series',)

In [25]:
avg.__closure__ # Each item in avg.__closure__ corresponds to a name in avg.__code__.co_freevars. These items are cells, and they have an attribute called cell_contents where the actual value can be found

(<cell at 0x000002733FBF1EE0: list object at 0x0000027342BFDD80>,)

In [26]:
avg.__closure__[0].cell_contents

[10, 11]

### The nonlocal Declaration

In [29]:
def make_averager():
    count = 0
    total = 0

    def averager(new_value):
        nonlocal count, total
        count += 1
        total += new_value
        return total/count 
    return averager

In [31]:
avg = make_averager()
avg(10)

10.0

### Implementing a Simple Decorator

In [2]:
import time
def clock(func):
    def clocked(*args):
        t0 = time.perf_counter()
        result = func(*args)
        elapsed = time.perf_counter()
        name = func.__name__
        arg_str = ', '.join(repr(arg) for arg in args)
        print('[%0.8fs] %s(%s) -> %r' % (elapsed, name, arg_str,result))
        return result
    return clocked # Return the inner function to replace the decorated function 

In [3]:
@clock
def snooze(seconds):
    time.sleep(seconds)

In [4]:
snooze(.123)

[758.23744350s] snooze(0.123) -> None


In [5]:
snooze.__name__ #name is clocked

'clocked'

In [12]:
import time
import functools
def clock(func):
    @functools.wraps(func)
    def clocked(*args, **kwargs):
        t0 = time.time()
        result = func(*args, **kwargs)
        elapsed = time.time() - t0
        name = func.__name__
        arg_lst = []
        if args:
            arg_lst.append(', '.join(repr(arg) for arg in args))
        if kwargs:
            pairs = ['%s=%r' % (k, w) for k, w in sorted(kwargs.items())]
            arg_lst.append(', '.join(pairs))
        arg_str = ', '.join(arg_lst)
        print('[%0.8fs] %s(%s) -> %r ' % (elapsed, name, arg_str, result))
        return result
    return clocked

In [9]:
@clock
def snooze(seconds):
    time.sleep(seconds)

snooze.__name__ # name is snooze

'snooze'

### Memoization with functoools.lru_cache
lru stand for Least Recently Used, meaning that the growth of the cache is limited by discarding the entries that have not been read for a while

In [13]:
import functools 

@functools.lru_cache() # lru_cache must be invoked as a regular function. The reason is that it accepts configuration parameters
@clock
def fibonacci(n):
    if n<2:
        return n
    return fibonacci(n-2) + fibonacci(n-1)


In [14]:
fibonacci(6)

[0.00000000s] fibonacci(0) -> 0 
[0.00000000s] fibonacci(1) -> 1 
[0.00000000s] fibonacci(2) -> 1 
[0.00000000s] fibonacci(3) -> 2 
[0.00000000s] fibonacci(4) -> 3 
[0.00000000s] fibonacci(5) -> 5 
[0.00000000s] fibonacci(6) -> 8 


8

lru_cache's full signature is 

In [ ]:
functools.lru_cache(maxsize=128,typed=False)

The maxsize argument determine how many call results are stored. The typed argument, if set to True, stores results of different argument types separately, i.e., distinguishing between integer and float that are normally considered equal, like 1 and 1.0

### Generic Functions with Single Dispatch

In [15]:
from functools import singledispatch 
from collections import abc 
import numbers 
import html 

@singledispatch # @singledispatch marks the base function that handles the object type.
def htmlize(obj):
    content = html.escape(repr(obj))
    return '<pre>{}</pre>'.format(content)

@htmlize.register(str) # Each specialized function is decorated with @«base_function».register(«type»).
def _(text):           # The name of the specialized functions is irrelevant; _ is a good choice to make this clear.
    content = html.escape(text).replace('\n','<br>\n')
    return '<p>{}</p>'.format(content)

@htmlize.register(numbers.Integral)
def _(n):
    return '/<pre>{0} (0x{0:x})</pre>'.format(n)

@htmlize.register(tuple)
@htmlize.register(abc.MutableSequence) #You can stack several register decorators to support different types with the same function.
def _(seq):
    inner = '</li>\n<li>'.join(htmlize(item) for item in seq)
    return '<ul>\n<li>' + inner + '</li>\n</ul>'

### Parameterized Decorators 
The main point is that higher order func(decorator factory) returns decorator, which is the applied to decorated function.

In [16]:
import time 
DEFAULT_FMT = '[{elapsed:0.8f}s {name}({args}) -> {result}]'

def clock(fmt=DEFAULT_FMT):    #clock is our parameterized decorated factory
    def decorate(func):        #decorate is the actual decorator
        def clocked(*args):    #clocked wraps the decorated function
            t0 = time.time()
            result = func(*args) #result is the actual result of the decorated function
            elapsed = time.time() - t0 
            name = func.__name__
            args = ', '.join(repr(arg) for arg in args)
            result = repr(result)
            print(fmt.format(**locals())) #using **locals() here allows any local variable of clocked to be referenced in the fmt
            return result #clocked will replace the decorated function
        return clocked #decorate returns clocked
    return decorate


In [17]:
@clock()
def snooze(seconds):
    time.sleep(seconds)

snooze(.123)

[0.13355708s snooze(0.123) -> None]


'None'